In [1]:
import findspark
findspark.init()

In [5]:
from pyspark.sql import SparkSession

In [3]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [6]:
spark = SparkSession.builder.master("local[3]").config("spark.jars.packages","org.apache.spark:spark-sql-kafka-0-10_2.11:2.4.8").getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

In [5]:
spark

In [6]:
df = spark.readStream.format("kafka").option("kafka.bootstrap.servers", "localhost:9092")\
      .option("subscribe", "first").option("startingOffsets", "earliest").load()

In [7]:
df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [9]:
df1.printSchema()

root
 |-- value: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)



In [10]:
pair = df1.select(explode(split("value"," ")).alias("words"),"timestamp")

In [11]:
pair.printSchema()

root
 |-- words: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)



In [12]:
wc = pair.groupby(window("timestamp",'30 second',"5 second"),"words").count()

In [14]:
result = wc.select("window.start","window.end","words","count")

In [13]:
wc.printSchema()

root
 |-- window: struct (nullable = true)
 |    |-- start: timestamp (nullable = true)
 |    |-- end: timestamp (nullable = true)
 |-- words: string (nullable = true)
 |-- count: long (nullable = false)



In [ ]:
result.writeStream.format("console").outputMode("update").start().awaitTermination()

3